# Sql and Python

One of the primary purpose of studying SQL for a data scientist or an ml engineer is to be able to query data from the database and create some kind of analysis. Some times though, one may need to do an analysis that would require more capabilities than what sql can offer, eg creating visualizations, or sending the processed data over to some kind of api service etc. This then necessitates the need to be able to access data from sql using a programming language that can do things such as visualizatons, ml, api requests etc. 

Another reason one may want to know about connecting with sql through python is that many times one needs to create databases!! Suppose you have data inside json files, you can't ingest this data in a database. You will need to process the json first and only then it can be fed into a db. 

The whole discussion on python-sql will revolve around two things:

1. Connecting to an existing sql database and fetching data into python
2. Creating a new database and populatiing it with tables using python


## Connecting to an existing database and fetching data

We can use `sqlalchemy` to connect to different kinds of databases. Its an ORM, heavily used by web-developers as well as data engineering folks.

In [1]:
from sqlalchemy.ext.automap import automap_base 
from sqlalchemy import create_engine
Base = automap_base() 
engine = create_engine('sqlite:///../../data/music.db')

In [2]:
## Automap lets you look at the tables in a db
Base.prepare(engine, reflect=True)
Base.classes.keys()

['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'Track',
 'MediaType',
 'Playlist']

In [3]:
## One can use the engine object to query data even without using Automap
data = engine.execute("Select * from Artist limit 5;")

In [4]:
data.fetchall()

[(1, 'AC/DC'),
 (2, 'Accept'),
 (3, 'Aerosmith'),
 (4, 'Alanis Morissette'),
 (5, 'Alice In Chains')]

In [5]:
 engine.execute("Select * from Employee limit 5;").fetchall()

[(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com'),
 (2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2002-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com'),
 (3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2002-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com'),
 (4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2003-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com'),
 (5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2003-10-17 00:00:00', '7727B 41 Ave', 'Cal

In [6]:
## But using automap one can access metadata such as column names
Artist = Base.classes.Artist
Album = Base.classes.Album

In [7]:
Artist.__table__.columns.keys()

['ArtistId', 'Name']

In [8]:
## One can use another api as well to query data (this is a very popular method while developing web-applications
## feel free to read more about query types here https://docs.sqlalchemy.org/en/14/orm/tutorial.html#querying
from sqlalchemy.orm import Session
session = Session(engine)
for artist in session.query(Artist).limit(10):
    print(artist.ArtistId, artist.Name)

1 AC/DC
2 Accept
3 Aerosmith
4 Alanis Morissette
5 Alice In Chains
6 Antônio Carlos Jobim
7 Apocalyptica
8 Audioslave
9 BackBeat
10 Billy Cobham


In [9]:
### What if you want the data in a pandas dataframe?
import pandas as pd
df_sql = pd.read_sql("Select * from artist;",engine)

In [10]:
df_sql

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains
...,...,...
270,271,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
271,272,Emerson String Quartet
272,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
273,274,Nash Ensemble


## Creating a db and ingesting flat files as tables.

The second task that can be handled effectively by python-sql is the creation of databases and tables within them.

We will learn the following:

1. Ingesting single table in a db from a flat file
2. Ingesting multiple related tables in a db from flat files

The process of table creation involves the following steps:

1. Defining the column types of the table
2. Defining any primary key/ foreign key constraints
3. Ingesting the data from the flat file to the table(s) in db

We will use the `retail_sales.csv` file and see how it can be ingested in the db

In [11]:
from sqlalchemy import Column, Integer, Numeric, String, DateTime ## for defining schema
from sqlalchemy.ext.declarative import declarative_base ## defining tables
from sqlalchemy import create_engine ## db path etc
from sqlalchemy.orm import Session ## for querying/inserting in db

In [12]:
Base = declarative_base()

In [13]:
class Sales(Base):
    __tablename__='sales'
    id = Column(Integer(),autoincrement=True,primary_key=True)
    sales_month = Column(DateTime())
    naics_code = Column(String())
    kind_of_business = Column(String())
    reason_for_null = Column(String())
    sales = Column(Numeric())       

In [14]:
engine = create_engine('sqlite:///../../data/example.db')

In [15]:
Base.metadata.create_all(engine) ## We can check if the table was created in the db

In [16]:
### Now lets create a session object so that we can insert the data in the table
session = Session(engine)

In [17]:
### read the data from csv
import csv
import datetime
cnt = 0
with open("../../data/retail_sales_us.csv","r",encoding='utf-8-sig') as f:
    reader = csv.DictReader(f,delimiter=",")
    for row in reader:
        db_row = Sales(sales_month = datetime.datetime.fromisoformat(row['sales_month']),
                       naics_code = row['naics_code'],
                       kind_of_business = row['kind_of_business'],
                       reason_for_null = row['reason_for_null'],
                       sales = row['sales'])
        session.add(db_row)
        try:
            session.commit()
        except:
            session.rollback()
        finally:
            cnt+=1
            if cnt>=1:
                break
   

In [18]:
## Lets insert all the rows:
engine = create_engine('sqlite:///../../data/example.db')
Base.metadata.create_all(engine)
session = Session(engine)

with open("../../data/retail_sales_us.csv","r",encoding='utf-8-sig') as f:
    reader = csv.DictReader(f,delimiter=",")
    for row in reader:
        db_row = Sales(sales_month = datetime.datetime.fromisoformat(row['sales_month']),
                       naics_code = row['naics_code'],
                       kind_of_business = row['kind_of_business'],
                       reason_for_null = row['reason_for_null'],
                       sales = row['sales'])
        session.add(db_row)
        try:
            session.commit()
        except:
            session.rollback()

## Inserting data when multiple tables are related

Lets use the candidates and contributions flat files and ingest them in the db. Also to be noted is the fact that both the tables are related. 

- Candidates will become the parent table
- Contributions will becoms the child table

In [29]:
from sqlalchemy.orm import relationship
from sqlalchemy import ForeignKey
Base = declarative_base()

class Candidates(Base):
    __tablename__ = 'candidates'
    
    candidate_id = Column(Integer(),primary_key=True)
    first_name = Column(String(),nullable=False)
    last_name = Column(String(),nullable=False)
    middle_name = Column(String())
    party = Column(String(),nullable=False)
    
class Contributions(Base):
    __tablename__ = "contributions"
    
    id = Column(Integer(),autoincrement=True,primary_key=True)
    first_name = Column(String(),nullable=False)
    last_name = Column(String(),nullable=False)
    middle_name = Column(String()) 
    street_1 = Column(String()) 
    street_2 = Column(String()) 
    city = Column(String()) 
    state = Column(String()) 
    zip = Column(String()) 
    amount = Column(Numeric())
    date = Column(DateTime())
    candidate_id = Column(Integer(),ForeignKey('candidates.candidate_id'))
    
engine = create_engine('sqlite:///../../data/relationship.db')
Base.metadata.create_all(engine)   

In [30]:
session = Session(engine)
with open("../../../Week1/data/candidates.csv","r") as f:
    reader = csv.DictReader(f,delimiter = ",")
    for row in reader:
        candidate = Candidates(candidate_id = row['id'],
                              first_name = row['first_name'],
                              last_name = row['last_name'],
                              middle_name = row['middle_name'],
                              party = row['party'])
        session.add(candidate)
        try:
            session.commit()
        except:
            session.rollback()

In [33]:
with open("../../../Week1/data/contributions.csv","r") as f:
    reader = csv.DictReader(f,delimiter = ",")
    for row in reader:
        contribution = Contributions(first_name = row['first_name'],
                                     last_name = row['last_name'],
                                     middle_name = row['middle_name'],
                                     street_1 = row['street_1'],
                                     street_2 = row['street_2'],
                                     city = row['city'],
                                     state = row['state'],
                                     zip = row['zip'],
                                     amount = row['amount'],
                                     date = datetime.date.fromisoformat(row['date']),
                                     candidate_id = row['candidate_id'])
        session.add(contribution)
        try:
            session.commit()
        except:
            session.rollback()
                                     